In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique')
import magellan as mg
mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


True

In [2]:
A = mg.load_dataset('table_A')
B = mg.load_dataset('table_B')

time: 4.97 ms


In [7]:
A['dummy'] = 1
B['dummy'] = 1

time: 1.89 ms


In [3]:
feature_table = mg.get_features_for_blocking(A, B)

time: 16.7 ms


In [19]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'dummy', 'dummy', 'zipcode', 'zipcode')

time: 8.19 ms


In [23]:
fv = mg.extract_feat_vecs(C, attrs_before=['ltable.zipcode', 'rtable.zipcode'],
                                           feat_table=feature_table)

time: 118 ms


In [54]:
A

,ID,name,birth_year,hourly_wage,address,zipcode,dummy
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107,1
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122,1
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107,1
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122,1
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122,1


time: 14.8 ms


In [24]:
exp = fv.ix[((fv.name_name_mel >= 0.4) | 
     (fv.birth_year_birth_year_lev >= 0.5)) &
     (fv.zipcode_zipcode_exm == 1), 
     ['ltable.ID', 'rtable.ID', 'ltable.zipcode', 'rtable.zipcode']]

time: 5.23 ms


In [55]:
B

,ID,name,birth_year,hourly_wage,address,zipcode,dummy
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107,1
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107,1
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122,1
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122,1
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122,1
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107,1


time: 15.2 ms


In [43]:
rb = mg.RuleBasedBlocker()
rb.add_rule(['name_name_mel(ltuple, rtuple) < 0.4',
                 'birth_year_birth_year_lev(ltuple, rtuple) < 0.5'],
                feature_table)
rb.add_rule(['zipcode_zipcode_exm(ltuple, rtuple) == 0'],
                feature_table)
D = rb.block_tables(A, B, 'zipcode', 'zipcode')

time: 52.3 ms


In [40]:
rb.rule_source['_rule_0']

'def _rule_0(ltuple, rtuple):\n    return name_name_mel(ltuple, rtuple) < 0.4 and birth_year_birth_year_lev(ltuple, rtuple) < 0.5'

time: 2.07 ms


In [42]:
x = rb.rules['_rule_0'](A.ix[0], B.ix[0]) 
print x
y = rb.rules['_rule_1'](A.ix[0], B.ix[0])
print y

False
False
time: 3.73 ms


In [44]:
D

,_id,ltable.ID,rtable.ID,ltable.zipcode,rtable.zipcode
0,0,a1,b1,94107,94107
1,1,a1,b2,94107,94107
2,2,a1,b6,94107,94107
3,3,a2,b3,94122,94122
4,4,a2,b4,94122,94122
5,5,a2,b5,94122,94122
6,6,a3,b1,94107,94107
7,7,a3,b2,94107,94107
8,8,a3,b6,94107,94107
9,9,a4,b3,94122,94122


time: 18.7 ms


In [49]:
e1 = list(exp.set_index(['ltable.ID', 'rtable.ID']).index.values)

time: 2.72 ms


In [52]:
e2 = list(D.set_index(['ltable.ID', 'ltable.zipcode']).index.values)

time: 5.11 ms


In [56]:
from magellan.feature.tokenizers import *

time: 1.08 ms


In [61]:
mg._global_sim_fns

,function_name,short_name
0,jaccard,jac
1,lev,lev
2,cosine,cos
3,monge_elkan,mel
4,needleman_wunsch,nmw
5,smith_waterman,sw
6,smith_waterman_gotoh,swg
7,jaro,jar
8,jaro_winkler,jwn
9,soundex,sdx


time: 10.4 ms
